In [1]:
!pip install language_tool_python --upgrade
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import language_tool_python
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

In [3]:
data_path = '../case_study_examples.csv'
df = pd.read_csv(data_path)

In [4]:
df.head()

,USE_similarity,dataset,original_class,original_text,perturbed_class,perturbed_text,run_type
0,0.930254,movie_review,0,davis is so enamored of her own creation that ...,1,davis is even enamored of her own creation tha...,jin
1,0.808168,movie_review,0,"somewhere in the middle, the film compels, as ...",1,"somewhere in the eastwards, the film commits, ...",jin
2,0.930277,movie_review,1,"a muckraking job, the cinematic equivalent of ...",0,"a muckraking job, the cinematic equivalent of ...",jin
3,0.951098,movie_review,0,it's hard to understand why anyone in his righ...,1,it's intense to understand why anyone in his r...,jin
4,0.912998,movie_review,0,never having seen the first two films in the s...,1,never having seen the first two cinema in the ...,jin


In [5]:
lang_tool = language_tool_python.LanguageTool("en-US")

Unzipping C:\Users\user\AppData\Local\Temp\tmp8b54u68u.zip to C:\Users\user\.cache\language_tool_python.
Downloaded https://www.languagetool.org/download/LanguageTool-6.4.zip to C:\Users\user\.cache\language_tool_python.


In [6]:
def get_errors(text):
    return lang_tool.check(text)

def get_error_count(text):
    return len(get_errors(text))

In [7]:
df['original_text_errors'] = df['original_text'].progress_map(get_errors)
df['perturbed_text_errors'] = df['perturbed_text'].progress_map(get_errors)

100%|███████████████| 3115/3115 [22:00<00:00,  2.36it/s]


In [8]:
df['original_text_error_count'] = df['original_text_errors'].map(len)
df['perturbed_text_error_count'] = df['perturbed_text_errors'].map(len)

In [9]:
original_text_errors = [e for l in df['original_text_errors'] for e in l]
perturbed_text_errors = [e for l in df['perturbed_text_errors'] for e in l]
print('Original errors:', len(original_text_errors))
print('Perturbed errors:', len(perturbed_text_errors))

Original errors: 48054
Perturbed errors: 51187


In [10]:
print('Added errors:', (df['original_text_error_count'] < df['perturbed_text_error_count']).sum())
print('Same # errors:', (df['original_text_error_count'] == df['perturbed_text_error_count']).sum())
print('Removed errors:', (df['original_text_error_count'] > df['perturbed_text_error_count']).sum())

Added errors: 1438
Same # errors: 1465
Removed errors: 212


In [11]:
df['run'] = df['dataset'] + '_' + df['run_type']
print(df['run'].unique())

['movie_review_jin' 'yelp_polarity_jin' 'imdb_jin' 'imdb_alzantot']


In [12]:
original_text_error_rule_ids = pd.Series([x.ruleId for x in original_text_errors])
perturbed_text_error_rule_ids = pd.Series([x.ruleId for x in perturbed_text_errors])

original_text_error_rule_freqs = original_text_error_rule_ids.value_counts().to_dict()
perturbed_text_error_rule_freqs = perturbed_text_error_rule_ids.value_counts().to_dict()

print('Original errors:', original_text_error_rule_ids.value_counts())
print()
print('Perturbed errors:', perturbed_text_error_rule_ids.value_counts())

Original errors: MORFOLOGIK_RULE_EN_US           18449
UPPERCASE_SENTENCE_START         9716
I_LOWERCASE                      8997
COMMA_PARENTHESIS_WHITESPACE     3919
COMMA_COMPOUND_SENTENCE           762
                                ...  
EN_COMPOUNDS_WELL_FORMED            1
EN_COMPOUNDS_WORLD_WEARY            1
EN_COMPOUNDS_LIP_SYNCHING           1
RATHER_THEN                         1
EN_COMPOUNDS_HIGH_SPIRITED          1
Name: count, Length: 1007, dtype: int64

Perturbed errors: MORFOLOGIK_RULE_EN_US            19669
UPPERCASE_SENTENCE_START          9751
I_LOWERCASE                       8693
COMMA_PARENTHESIS_WHITESPACE      3920
COMMA_COMPOUND_SENTENCE            731
                                 ...  
SHE_LIVE                             1
EN_COMPOUNDS_ANTI_VIRUS              1
ROLLS_ROYCE                          1
EN_COMPOUNDS_POVERTY_STRICKEN        1
DONTCHA                              1
Name: count, Length: 1043, dtype: int64


In [13]:
err_data = []

for rule_id, pert_freq in perturbed_text_error_rule_freqs.items():
    orig_freq = original_text_error_rule_freqs.get(rule_id, 0)
    freq_diff = pert_freq - orig_freq
    freq_ratio = pert_freq / orig_freq if orig_freq else None
    err_data.append({
        'freq_diff':  freq_diff,
        'orig_freq':  orig_freq,
        'pert_freq':  pert_freq,
        'freq_ratio': freq_ratio,
        'rule_id': rule_id
    })
    
err_data = pd.DataFrame(err_data)
err_data['freq_diff_perc'] = err_data['freq_diff'] / float(len(df))
err_data.head()

,freq_diff,orig_freq,pert_freq,freq_ratio,rule_id,freq_diff_perc
0,1220,18449,19669,1.066128,MORFOLOGIK_RULE_EN_US,0.391653
1,35,9716,9751,1.003602,UPPERCASE_SENTENCE_START,0.011236
2,-304,8997,8693,0.966211,I_LOWERCASE,-0.097592
3,1,3919,3920,1.000255,COMMA_PARENTHESIS_WHITESPACE,0.000321
4,-31,762,731,0.959318,COMMA_COMPOUND_SENTENCE,-0.009952


In [14]:
len(df)

3115

In [15]:
pd.set_option('display.max_rows', None)

err_data.sort_values(by='freq_diff', ascending=False)

,freq_diff,orig_freq,pert_freq,freq_ratio,rule_id,freq_diff_perc
0,1220,18449,19669,1.066128,MORFOLOGIK_RULE_EN_US,0.391653
6,369,98,467,4.765306,EN_A_VS_AN,0.118459
8,352,28,380,13.571429,A_NNS,0.113002
10,245,13,258,19.846154,MD_BASEFORM,0.078652
12,138,88,226,2.568182,EN_UNPAIRED_QUOTES,0.044302
17,109,3,112,37.333333,PRP_VBG,0.034992
15,106,14,120,8.571429,THIS_NNS,0.034029
21,84,2,86,43.000000,TO_NON_BASE,0.026966
25,75,4,79,19.750000,NON3PRS_VERB,0.024077
19,67,21,88,4.190476,BEEN_PART_AGREEMENT,0.021509


In [16]:
err_data[err_data['freq_ratio'].isna()].sort_values(by='freq_diff', ascending=False)

,freq_diff,orig_freq,pert_freq,freq_ratio,rule_id,freq_diff_perc
49,29,0,29,NaN,ARTICLE_ADJECTIVE_OF,0.009310
70,19,0,19,NaN,WANT_TO_NN,0.006100
97,14,0,14,NaN,AND_BUT,0.004494
105,12,0,12,NaN,BESTEST,0.003852
122,10,0,10,NaN,WHAT_TO_VBD,0.003210
123,10,0,10,NaN,DONT_NEEDS,0.003210
142,9,0,9,NaN,COULDVE_IRREGULAR_VERB,0.002889
167,7,0,7,NaN,DIDN_T_BEEN_SOLVED,0.002247
177,6,0,6,NaN,MENTION_ABOUT,0.001926
184,6,0,6,NaN,BE_TO_VBG,0.001926


In [17]:
err_data.sort_values(by=['pert_freq'], ascending=False)

,freq_diff,orig_freq,pert_freq,freq_ratio,rule_id,freq_diff_perc
0,1152,20147,21299,1.057180,MORFOLOGIK_RULE_EN_US,0.369823
1,46,9368,9414,1.004910,UPPERCASE_SENTENCE_START,0.014767
2,-307,9041,8734,0.966044,I_LOWERCASE,-0.098555
3,3,3853,3856,1.000779,COMMA_PARENTHESIS_WHITESPACE,0.000963
4,0,1818,1818,1.000000,EN_QUOTES,0.000000
5,403,99,502,5.070707,EN_A_VS_AN,0.129374
6,-17,450,433,0.962222,COMMA_COMPOUND_SENTENCE,-0.005457
7,0,428,428,1.000000,PUNCTUATION_PARAGRAPH_END,0.000000
8,-65,491,426,0.867617,EN_SPECIFIC_CASE,-0.020867
9,-91,508,417,0.820866,EN_COMPOUNDS,-0.029213


In [ ]:
len(df)

In [19]:
err_data.sort_values(by=['freq_ratio', 'freq_diff'], ascending=False)

,freq_diff,orig_freq,pert_freq,freq_ratio,rule_id,freq_diff_perc
21,84,2,86,43.000000,TO_NON_BASE,0.026966
17,109,3,112,37.333333,PRP_VBG,0.034992
31,46,2,48,24.000000,PRP_MD_NN,0.014767
58,23,1,24,24.000000,PRP_VB,0.007384
10,245,13,258,19.846154,MD_BASEFORM,0.078652
25,75,4,79,19.750000,NON3PRS_VERB,0.024077
27,56,4,60,15.000000,AGREEMENT_SENT_START,0.017978
90,14,1,15,15.000000,NOUN_VERB_CONFUSION,0.004494
8,352,28,380,13.571429,A_NNS,0.113002
107,10,1,11,11.000000,PRP_JJ,0.003210


In [20]:
perturbed_text_errors[0]

Match({'ruleId': 'UPPERCASE_SENTENCE_START', 'message': 'This sentence does not start with an uppercase letter.', 'replacements': ['Davis'], 'offsetInContext': 0, 'context': 'davis is even enamored of her own creation th...', 'offset': 0, 'errorLength': 5, 'category': 'CASING', 'ruleIssueType': 'typographical', 'sentence': "davis is even enamored of her own creation that she can't suppose how intolerable the character is"})

In [21]:
for err in perturbed_text_errors:
    if err.ruleId == 'MORFOLOGIK_RULE_EN_US':
        badword = err.context[err.offset:err.offset+err.errorLength]
        goodwords = err.replacements
        print(err.message + ':', 'Replace',badword,'with one of',goodwords)
        print(err.category)
        print()
        break

Possible spelling mistake found.: Replace exper with one of ['lemme']
TYPOS



In [23]:
def yield_errs(err_code, num_to_print=1):
    n = 0
    for err in perturbed_text_errors:
        if n >= num_to_print: 
            break
        if err.ruleId == err_code: 
            print('err:', err)
            badword = err.context[err.offset:err.offset+err.errorLength]
            goodwords = err.replacements
            err_str = err_code + ': ' + err.message + ' || Replace '+badword+' with one of ' + '[' +','.join(goodwords) + ']'
            context = err.context
            cat = err.category
            yield err_str
            n += 1

def get_err(err_code):
    ans = list(yield_errs(err_code, num_to_print=1))
    return ans[0] if len(ans) else None

def get_errs(*args):
    return list(yield_errs(*args))

def print_errs(*args):
    print(get_errs(*args))

print_errs('I_AM')

[]


In [32]:
def yield_errs_context(err_code, num_to_print=1):
    n = 0
    for err in perturbed_text_errors:
        if n >= num_to_print: 
            break
        if err.ruleId == err_code: 
            badword = err.context[err.offset:err.offset+err.errorLength]
            goodwords = err.replacements
            err_str = err_code + ': ' + err.message + ' || Replace '+badword+' with one of ' + '[' +','.join(goodwords) + ']'
            context = err.context
            cat = err.category
            yield context
            n += 1

def get_err_context(err_code):
    ans = list(yield_errs_context(err_code, num_to_print=1))
    return ans[0] if len(ans) else None

def get_errs_context(*args):
    return list(yield_errs_context(*args))

def print_errs_context(*args):
    print(get_errs_context(*args))

print_errs('I_AM')

[]


In [24]:
get_err('DID_BASEFORM')

err: Offset 176, length 10, Rule ID: DID_BASEFORM
Message: The auxiliary verb ‘do’ requires the base form of the verb.
Suggestion: consider
...wishing for a spot to sittin down i did considered better eating come out of a academia do...
                                           ^^^^^^^^^^


'DID_BASEFORM: The auxiliary verb ‘do’ requires the base form of the verb. || Replace  with one of [consider]'

In [25]:
get_errs('DID_BASEFORM', 20)

err: Offset 176, length 10, Rule ID: DID_BASEFORM
Message: The auxiliary verb ‘do’ requires the base form of the verb.
Suggestion: consider
...wishing for a spot to sittin down i did considered better eating come out of a academia do...
                                           ^^^^^^^^^^
err: Offset 301, length 6, Rule ID: DID_BASEFORM
Message: The auxiliary verb ‘do’ requires the base form of the verb.
Suggestion: fool
...y attention to the food but mix did not fooled
                                           ^^^^^^
err: Offset 308, length 5, Rule ID: DID_BASEFORM
Message: The auxiliary verb ‘do’ requires the base form of the verb.
Suggestion: suit
...u're go to joe's lobsters shack did not suits down for you're uneasy the crabs mm on ...
                                           ^^^^^
err: Offset 602, length 4, Rule ID: DID_BASEFORM
Message: The auxiliary verb ‘do’ requires the base form of the verb.
Suggestion: say
... she overlook the bacon and didn't even said excuses, the ham

['DID_BASEFORM: The auxiliary verb ‘do’ requires the base form of the verb. || Replace  with one of [consider]',
 'DID_BASEFORM: The auxiliary verb ‘do’ requires the base form of the verb. || Replace  with one of [fool]',
 'DID_BASEFORM: The auxiliary verb ‘do’ requires the base form of the verb. || Replace  with one of [suit]',
 'DID_BASEFORM: The auxiliary verb ‘do’ requires the base form of the verb. || Replace  with one of [say]',
 'DID_BASEFORM: The auxiliary verb ‘do’ requires the base form of the verb. || Replace  with one of [discover]',
 'DID_BASEFORM: The auxiliary verb ‘do’ requires the base form of the verb. || Replace  with one of [believe]',
 'DID_BASEFORM: The auxiliary verb ‘do’ requires the base form of the verb. || Replace  with one of [rip]',
 'DID_BASEFORM: The auxiliary verb ‘do’ requires the base form of the verb. || Replace  with one of [manage]',
 'DID_BASEFORM: The auxiliary verb ‘do’ requires the base form of the verb. || Replace  with one of [demand]',
 'DID_

In [35]:
pd.options.display.max_colwidth = 500
err_data['Explanation'] = err_data['rule_id'].map(get_err)
err_data['Context'] = err_data['rule_id'].map(get_err_context)
err_data.head()

err: Offset 49, length 5, Rule ID: MORFOLOGIK_RULE_EN_US
Message: Possible spelling mistake found.
Suggestion: lemme
... in the eastwards, the film commits, as demme experimentation he captures a few scene...
                                                 ^^^^^
err: Offset 0, length 5, Rule ID: UPPERCASE_SENTENCE_START
Message: This sentence does not start with an uppercase letter
Suggestion: Davis
davis is even enamored of her own creation th...
^^^^^
err: Offset 54, length 1, Rule ID: I_LOWERCASE
Message: Did you mean "I"?
Suggestion: I
...een the first two cinema in the series, i can't compares friday after next to the...
                                                      ^
err: Offset 10, length 2, Rule ID: COMMA_PARENTHESIS_WHITESPACE
Message: Don't put a space after the opening parenthesis
Suggestion: (
rosenthal ( halloween iii ) sounds to recieve ignore...
          ^^
err: Offset 220, length 1, Rule ID: EN_QUOTES
Message: Use a smart opening quote here: "“".
Suggestion: “

err: Offset 682, length 8, Rule ID: MANY_NN_U
Message: Possible agreement error. The noun may seems to be uncountable; consider using: "much may", "a good deal of may".
Suggestion: much may; a good deal of may
...gs ( flipping cars and head squashing ) many may think it sound like utter crap that onl...
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          ^^^^^^^^
err:

err: Offset 682, length 6, Rule ID: AFFECT_EFFECT
Message: Did you mean "effect"?
Suggestion: effect
...fferent goal. it has an almost dizzying affect on you but at the same time is exciting...
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          ^^^^^^
err: Offset 460, length 8, Rule ID: CA_BUG_EYED
Message: 'bug eyed' seems to be a compound adjective in front of a no

,freq_diff,orig_freq,pert_freq,freq_ratio,rule_id,freq_diff_perc,Explanation,Context
0,1152,20147,21299,1.057180,MORFOLOGIK_RULE_EN_US,0.369823,MORFOLOGIK_RULE_EN_US: Possible spelling mistake found. || Replace exper with one of [lemme],"... in the eastwards, the film commits, as demme experimentation he captures a few scene..."
1,46,9368,9414,1.004910,UPPERCASE_SENTENCE_START,0.014767,UPPERCASE_SENTENCE_START: This sentence does not start with an uppercase letter || Replace davis with one of [Davis],davis is even enamored of her own creation th...
2,-307,9041,8734,0.966044,I_LOWERCASE,-0.098555,"I_LOWERCASE: Did you mean ""I""? || Replace p with one of [I]","...een the first two cinema in the series, i can't compares friday after next to the..."
3,3,3853,3856,1.000779,COMMA_PARENTHESIS_WHITESPACE,0.000963,COMMA_PARENTHESIS_WHITESPACE: Don't put a space after the opening parenthesis || Replace ( with one of [(],rosenthal ( halloween iii ) sounds to recieve ignore...
4,0,1818,1818,1.000000,EN_QUOTES,0.000000,"EN_QUOTES: Use a smart opening quote here: ""“"". || Replace with one of [“]","...anctimonious lloyd c douglas, (he wrote ""the robe""), and already filmed in 1935 w..."


In [26]:
err_data.sort_values(by=['freq_ratio','freq_diff'], ascending=False)

,freq_diff,orig_freq,pert_freq,freq_ratio,rule_id,freq_diff_perc
21,84,2,86,43.000000,TO_NON_BASE,0.026966
17,109,3,112,37.333333,PRP_VBG,0.034992
31,46,2,48,24.000000,PRP_MD_NN,0.014767
58,23,1,24,24.000000,PRP_VB,0.007384
10,245,13,258,19.846154,MD_BASEFORM,0.078652
25,75,4,79,19.750000,NON3PRS_VERB,0.024077
27,56,4,60,15.000000,AGREEMENT_SENT_START,0.017978
90,14,1,15,15.000000,NOUN_VERB_CONFUSION,0.004494
8,352,28,380,13.571429,A_NNS,0.113002
107,10,1,11,11.000000,PRP_JJ,0.003210


In [29]:
min_pert_freq = 25
min_pert_ratio = 5.0


explanatory_df = err_data \
    [(err_data['pert_freq'] > min_pert_freq) & (err_data['freq_ratio'] > min_pert_ratio)] \
    .sort_values(by=['freq_ratio','freq_diff'], ascending=False) \
    .reset_index() \
    .drop(['freq_ratio', 'freq_diff', 'index', 'freq_diff_perc'], axis=1) 

explanatory_df = explanatory_df[['rule_id', 'orig_freq', 'pert_freq']] #, 'Explanation', 'Context'
explanatory_df.head()

,rule_id,orig_freq,pert_freq
0,TO_NON_BASE,2,86
1,PRP_VBG,3,112
2,PRP_MD_NN,2,48
3,MD_BASEFORM,13,258
4,NON3PRS_VERB,4,79


In [30]:
latex = explanatory_df.to_latex()
for line in latex.split('\n'):
    print(line)

\begin{tabular}{llrr}
\toprule
 & rule_id & orig_freq & pert_freq \\
\midrule
0 & TO_NON_BASE & 2 & 86 \\
1 & PRP_VBG & 3 & 112 \\
2 & PRP_MD_NN & 2 & 48 \\
3 & MD_BASEFORM & 13 & 258 \\
4 & NON3PRS_VERB & 4 & 79 \\
5 & AGREEMENT_SENT_START & 4 & 60 \\
6 & A_NNS & 28 & 380 \\
7 & AUXILIARY_DO_WITH_INCORRECT_VERB_FORM & 5 & 46 \\
8 & PCT_PLURAL_NOUN_SINGULAR_VERB_AGREEMENT & 3 & 27 \\
9 & THIS_NNS & 14 & 120 \\
10 & PRP_PAST_PART & 7 & 55 \\
\bottomrule
\end{tabular}



In [31]:
df['run'].unique()

array(['movie_review_jin', 'yelp_polarity_jin', 'imdb_jin',
       'imdb_alzantot'], dtype=object)

In [32]:
len(df)

3115

In [33]:
df['introduced_error'] = df['original_text_error_count'] < df['perturbed_text_error_count']
df.groupby('run')['introduced_error'].mean()

run
imdb_alzantot        0.349800
imdb_jin             0.552972
movie_review_jin     0.303779
yelp_polarity_jin    0.596239
Name: introduced_error, dtype: float64

In [34]:
df['introduced_error'].mean()

np.float64(0.46163723916532906)